In [9]:
import pandas as pd

from collections import Counter
from tqdm import tqdm_notebook

from litecoder import twitter_usa_city_state

from frontline.db import session
from frontline.models import Tweet

In [2]:
locs = [r for r, in session.query(Tweet.actor_location) if r]

In [3]:
len(locs)

24007

In [6]:
city_counts = Counter()
state_counts = Counter()

for loc in tqdm_notebook(locs):
    
    city, state = twitter_usa_city_state(loc)
    
    if city:
        city_counts[city] += 1
        
    if state:
        state_counts[state] += 1

HBox(children=(IntProgress(value=0, max=24007), HTML(value='')))

In [22]:
data = []
for city, count in city_counts.items():
    data.append((city.name, city.admin1_code, city.population, city.latitude, city.longitude, count))
    
cities = pd.DataFrame(data, columns=('city', 'state', 'pop', 'latitude', 'longitude', 'count'))

cities['count_pop'] = cities['count'] / cities['pop']

In [23]:
cities.sort_values('count', ascending=False).head(20)

,city,state,pop,latitude,longitude,count,count_pop
16,Chicago,IL,2720546,41.85003,-87.65005,456,0.000168
42,San Francisco,CA,864816,37.77493,-122.41942,453,0.000524
7,Los Angeles,CA,3971883,34.05223,-118.24368,450,0.000113
13,Boston,MA,667137,42.35843,-71.05977,327,0.000490
30,Seattle,WA,684451,47.60621,-122.33207,269,0.000393
0,Brooklyn,NY,2300664,40.65010,-73.94958,221,0.000096
46,San Diego,CA,1394928,32.71533,-117.15726,179,0.000128
37,New York City,NY,8175133,40.71427,-74.00597,171,0.000021
1,Atlanta,GA,463878,33.74900,-84.38798,153,0.000330
73,Houston,TX,2296224,29.76328,-95.36327,147,0.000064


In [24]:
cities[cities['pop'] > 1e5].sort_values('count_pop', ascending=False).head(20)

,city,state,pop,latitude,longitude,count,count_pop
650,Wilmington,NC,115933,34.22573,-77.94471,78,0.000673
42,San Francisco,CA,864816,37.77493,-122.41942,453,0.000524
34,Berkeley,CA,120972,37.87159,-122.27275,61,0.000504
13,Boston,MA,667137,42.35843,-71.05977,327,0.000490
71,Murfreesboro,TN,126118,35.84562,-86.39027,56,0.000444
30,Seattle,WA,684451,47.60621,-122.33207,269,0.000393
1,Atlanta,GA,463878,33.74900,-84.38798,153,0.000330
31,Wichita,KS,389965,37.69224,-97.33754,123,0.000315
19,Arlington,VA,207627,38.88101,-77.10428,64,0.000308
78,Miami,FL,441003,25.77427,-80.19366,126,0.000286


In [26]:
cities.to_csv('cities.csv')

In [29]:
state_pops = pd.read_csv('state-pops.csv')

In [38]:
data = []
for state, count in state_counts.items():
    pop = int(state_pops[state_pops['state'] == state.name]['pop_est_2014'])
    data.append((state.name, pop, count))
    
states = pd.DataFrame(data, columns=('state', 'pop', 'count'))

states['count_pop'] = states['count'] / states['pop']

In [39]:
states

,state,pop,count,count_pop
0,New York,19746227,844,0.000043
1,Pennsylvania,12787209,420,0.000033
2,Georgia,10097343,277,0.000027
3,Tennessee,6549352,209,0.000032
4,Illinois,12880580,619,0.000048
5,North Carolina,9943964,408,0.000041
6,Maryland,5976407,194,0.000032
7,Louisiana,4649676,164,0.000035
8,Texas,26956958,899,0.000033
9,California,38802500,2590,0.000067
